In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import cudf
import os
import glob

In this notebook, we perform exploratory data analysis (EDA) on the raw product metadata to gain a basic understanding of the dataset and its key characteristics. The raw dataset can be found from [here](https://drive.google.com/drive/folders/1l-CFy07rRqw9Hb2gOGAVR4NGc19Sjlcl).

In [ ]:
gdf = cudf.read_parquet('../../data/raw/raw_data.parquet')

### Missing Data Overview
In this section, we examine each column to identify the presence of missing values. Based on the results below, we find that `ShortDescription` and `Keyword`s are missing over 90% of their data. Additionally, several other columns also have a significant proportion of missing values.

In [4]:
# % of null rows
print(gdf.isnull().sum() / len(df) * 100)

Pid                  0.000000
Name                 0.223484
ShortDescription    91.055908
Description          0.029069
CategoryId          28.932255
Category            28.932255
ImageURL             0.005707
Price                0.000943
PriceCurrency        0.090815
SalePrice            4.649404
FinalPrice           0.000943
Discount             0.000000
isOnSale             0.000000
IsInStock            0.000000
Keywords            99.178080
Brand               53.043597
Manufacturer        34.154016
MPN                 53.870327
UPCorEAN            61.140028
SKU                  0.000000
Color               49.713821
Gender              54.632003
Size                46.570427
Condition           10.031526
dtype: float64


### Check the distinct value count for each column

In this step, we examine the number of unique values in each column to understand the variability and potential categorical nature of the data.  

Based on the results below, we found that total `CategoryId` is 3588, but not sure how much of these categories overlap.

In [5]:
gdf.nunique()

Pid                 15147805
Name                 9226783
ShortDescription     1064972
Description          5670091
CategoryId              3588
Category               15844
ImageURL             5753395
Price                 199462
PriceCurrency             34
SalePrice             162116
FinalPrice            163978
Discount                 101
isOnSale                   2
IsInStock                  2
Keywords                5090
Brand                  21247
Manufacturer           25883
MPN                  2350925
UPCorEAN             3541939
SKU                 10443422
Color                 170433
Gender                     3
Size                   55761
Condition                  5
dtype: int64

Largest cateogries are books and varity of apparels

In [6]:
gdf['Category'].value_counts(normalize=True, dropna=False) * 100

Category
<NA>                                                                                          28.932255
Media >Books                                                                                   7.358877
Apparel & Accessories >Shoes                                                                   6.447371
Apparel & Accessories >Clothing >Shirts & Tops                                                 5.735526
Apparel & Accessories                                                                          4.258397
                                                                                                ...    
Casa e giardino >Biancheria >Lenzuola e coperte >Mantovane per letto                           0.000007
Comercial e industrial >Ciência e laboratório >Suprimentos de laboratório >Placa de Petri      0.000007
Heimwerkerbedarf >Werkzeugzubehör >Klammern                                                    0.000007
Kunst & Unterhaltung >Hobby & Kunst >Musikinstrumente >

If we add `USD`, `CAD` and `GBP`, only about 70% of the data is guranteed to be in English. 

In [7]:
gdf['PriceCurrency'].value_counts(normalize=True, dropna=False) * 100

PriceCurrency
USD         60.797999
EUR         11.729155
CAD          9.201871
GBP          1.667728
SEK          1.470434
CHF          1.266301
ZAR          1.189462
BRL          1.081857
AUD          1.071736
JPY          1.068454
COP          1.032319
CLP          1.008502
KRW          0.994007
DKK          0.886994
NOK          0.861779
ARS          0.630151
MXN          0.476498
SAR          0.465825
PEN          0.405880
NZD          0.390767
MYR          0.334183
AED          0.325199
KWD          0.280549
CNY          0.222756
INR          0.220468
TRY          0.216626
RUB          0.185978
PHP          0.174089
SGD          0.114254
<NA>         0.090815
THB          0.070950
HKD          0.023284
TWD          0.022016
PLN          0.021048
0.00 USD     0.000065
Name: proportion, dtype: float64

53% of products don't have `Brand` information. The brands are also not distributed evenly.

In [8]:
gdf['Brand'].value_counts(normalize=True, dropna=False) * 100

Brand
<NA>                               53.043597
Lands' End                          0.997601
Nike                                0.639777
Levi's                              0.496545
Ray-Ban                             0.427071
                                     ...    
BEATRICE BOYLE for BROWNS FOCUS     0.000007
Esschert Design                     0.000007
Mighty Sesame Co                    0.000007
SARAH KUHLMANN SWIMWEAR             0.000007
Yeast Gard                          0.000007
Name: proportion, Length: 21248, dtype: float64

Temu is labelled as a manufacturer even though it's an ecommerce site. Not sure how these data overlap with `Brand`. 

In [9]:
gdf['Manufacturer'].value_counts(normalize=True, dropna=False) * 100

Manufacturer
<NA>                       34.154016
Temu                       13.930955
Lands' End                  0.997601
Nike                        0.873811
Levi's                      0.499061
                             ...    
Esschert Design             0.000007
Mighty Sesame Co            0.000007
SARAH KUHLMANN SWIMWEAR     0.000007
OzCut                       0.000007
Yeast Gard                  0.000007
Name: proportion, Length: 25884, dtype: float64

In [10]:
gdf['Color'].value_counts(normalize=True, dropna=False) * 100

Color
<NA>                   49.713821
Black                   8.038111
White                   2.272229
Blue                    1.528520
Red                     0.908171
                         ...    
Spicy Melon             0.000007
Matte Pink/Smoke        0.000007
Pinon Green             0.000007
Navy Baby               0.000007
Light Gray Branches     0.000007
Name: proportion, Length: 170434, dtype: float64

In [11]:
gdf['Gender'].value_counts(normalize=True, dropna=False) * 100

Gender
<NA>      54.632003
female    24.382343
male      15.222691
unisex     5.762963
Name: proportion, dtype: float64

In [12]:
gdf['Size'].value_counts(normalize=True, dropna=False) * 100

Size
<NA>                     46.570427
01                        1.761429
M                         1.600672
S                         1.570817
L                         1.554807
                           ...    
33.5in                    0.000007
Makes 20 lbs              0.000007
56P x 56L x 76H cm        0.000007
39*78 In                  0.000007
40X40cm/15.7X15.7inch     0.000007
Name: proportion, Length: 55762, dtype: float64

In [13]:
gdf['Condition'].value_counts(normalize=True, dropna=False) * 100

Condition
new            37.890660
New            33.695159
Used           18.375940
<NA>           10.031526
refurbished     0.005733
Refurbished     0.000982
Name: proportion, dtype: float64